
<a href="https://colab.research.google.com/github/is-leeroy-jenkins/Halo-Kitty-Adventures/blob/main/sql/notebooks/access.ipynb" target="_parent">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🗂️ SQL: MS Access



*A Deep Dive into Jet/ACE SQL and VBA Integration*

## 🧭 Introduction

- Microsoft Access is not just a spreadsheet replacement — it’s a **relational database system** that uses a version of SQL known as **Jet/ACE SQL**.

- While the SQL syntax in Access looks similar to SQL Server or MySQL, it has unique behavior, functions, and data-type handling rules because it’s interpreted by the **Microsoft Access Database Engine (ACE)**.

- Access stores all database tables, queries, forms, reports, macros, and modules in the Access Jet database as a single file.

- For query development, Access offers a `Query Designer`, a graphical user interface that allows users to build queries without knowledge of structured query language. In the Query Designer, users can "show" the datasources of the query (which can be tables or queries) and select the fields they want returned by clicking and dragging them into the grid. One can set up joins by clicking and dragging fields in tables to fields in other tables. Access allows users to view and manipulate the SQL code if desired. Any Access table, including linked tables from different data sources, can be used in a query.

- Access also supports the creation of "pass-through queries". These snippets of SQL code can address external data sources through the use of ODBC connections on the local machine. This enables users to interact with data stored outside the Access program without using linked tables or Jet. Users construct the pass-through queries using the SQL syntax supported by the external data source.

- When developing reports (in "Design View") additions or changes to controls cause any linked queries to execute in the background and the designer is forced to wait for records to be returned before being able to make another change. This feature cannot be turned off.

- Non-programmers can use the macro feature to automate simple tasks through a series of drop-down selections. Macros allow users to easily chain commands together such as running queries, importing or exporting data, opening and closing forms, previewing and printing reports, etc. Macros support basic logic (IF-conditions) and the ability to call other macros. Macros can also contain sub-macros which are similar to subroutines. In Access 2007, enhanced macros included error-handling and support for temporary variables. 

- Access 2007 also introduced embedded macros that are essentially properties of an object's event. This eliminated the need to store macros as individual objects. However, macros were limited in their functionality by a lack of programming loops and advanced coding logic until Access 2013. 

- With significant further enhancements introduced in Access 2013, the capabilities of macros became fully comparable to VBA. They made feature rich web-based application deployments practical, via a greatly enhanced Microsoft SharePoint interface and tools, as well as on traditional Windows desktops



#### Split database architecture

- Microsoft Access applications can adopt a split-database architecture. The single database can be divided into a separate "back-end" file that contains the data tables (shared on a file server) and a "front-end" (containing the application's objects such as queries, forms, reports, macros, and modules). 

- The "front-end" Access application is distributed to each user's desktop and linked to the shared database. Using this approach, each user has a copy of Microsoft Access (or the runtime version) installed on their machine along with their application database. This reduces network traffic since the application is not retrieved for each use. 

- The "front-end" database can still contain local tables for storing a user's settings or temporary data. This split-database design also allows development of the application independent of the data. One disadvantage is that users may make various changes to their own local copy of the application and this makes it hard to manage version control. 

- When a new version is ready, the front-end database is replaced without impacting the data database. Microsoft Access has two built-in utilities, Database Splitter and Linked Table Manager, to facilitate this architecture.

#### This guide will teach you:

* How to write and understand SQL queries within Access

* How to execute queries using **VBA**

* How to combine Access forms, reports, and macros with SQL for automation and reporting

We’ll move gradually from basic query construction to advanced topics like parameter queries, joins, subqueries, and crosstab reports — all written in clean Access SQL.


## ⚙️ SQL Environment

### Where SQL Lives in Access

- [Download/Install](https://www.microsoft.com/en-us/download/details.aspx?id=54920)

Every Access database (`.accdb` or `.mdb`) has an underlying **database engine** (Jet for older versions, ACE for newer).
When you create a query in the **Query Design View**, Access actually builds an SQL statement behind the scenes.

You can view or edit that statement directly by switching to **SQL View**:

* Open the Query Designer.

* Select **View → SQL View** from the toolbar.

The **SQL View** window is where Access interprets and stores SQL commands.

## ✍️ SQL Queries


| Query Type               | Purpose                                 | Returns Results? |
| ------------------------ | --------------------------------------- | ---------------- |
| **SELECT**               | Retrieves data.                         | ✅ Yes            |
| **INSERT INTO**          | Adds new records.                       | ❌ No             |
| **UPDATE**               | Modifies existing records.              | ❌ No             |
| **DELETE**               | Removes records.                        | ❌ No             |
| **MAKE-TABLE**           | Creates a new table from query results. | ❌ No             |
| **APPEND**               | Adds data to an existing table.         | ❌ No             |
| **CROSSTAB (TRANSFORM)** | Summarizes data in pivot-table format.  | ✅ Yes            |
| **UNION**                | Combines multiple datasets.             | ✅ Yes            |



### SQL in VBA

- Access’s **VBA environment** (Visual Basic for Applications) gives you full control over executing SQL.

- Two main approaches exist:

    - 1. **DAO (Data Access Objects)** – the most direct interface to Access tables and queries.

    - 2. **DoCmd methods** – used for running saved queries or executing SQL strings directly.

> This dual environment — SQL inside Access and SQL inside VBA — is what makes Access both beginner-friendly and powerful for automation.

Example:


In [ ]:

' Run an action query (no results returned)
CurrentDb.Execute "UPDATE Employees SET Salary = Salary * 1.05;", dbFailOnError


In [ ]:
' Open a recordset and read data
Dim rs As DAO.Recordset
Set rs = CurrentDb.OpenRecordset("SELECT FirstName, LastName FROM Employees;")
Do While Not rs.EOF
    Debug.Print rs!FirstName, rs!LastName
    rs.MoveNext
Loop
rs.Close

### 🎯 Wildcards

- Built-in pattern matching provides a versatile tool for making string comparisons.

- You can use a group of one or more characters (charlist) enclosed in brackets `[ ]` to match any single character in expression, and charlist can include almost any characters in the ANSI character set, including digits. 

- You can use the special characters opening bracket `[`, question mark `?`, number sign `#`, and asterisk `*` to match themselves directly only if enclosed in brackets. 

- You cannot use the closing bracket `]` within a group to match itself, but you can use it outside a group as an individual character.

| Wildcard | Meaning                              | Example (Access SQL)           | Matches                 |
| -------- | ------------------------------------ | ------------------------------ | ----------------------- |
| `*`      | Any number of characters (0+ chars)  | `WHERE Name LIKE "A*"`         | Alice, Andrew, A, Adams |
| `?`      | Any single character                 | `WHERE Code LIKE "A?1"`        | A11, AB1                |
| `#`      | Any single digit (0–9)               | `WHERE Phone LIKE "###-###"`   | 123-456                 |
| `[abc]`  | Any **one** of the characters listed | `WHERE Code LIKE "[AB]12"`     | A12 or B12              |
| `[!abc]` | Any one character **not** listed     | `WHERE Code LIKE "[!A]12"`     | B12, C12, Z12           |
| `[a-z]`  | Character range                      | `WHERE LastName LIKE "[A-M]*"` | Names A–M               |
| `[!a-z]` | Any character **outside** the range  | `WHERE Code LIKE "[!0-9]*"`    | Non-numeric start       |


## 🔍 SELECT STATEMENT

- `SELECT` statements do not change data in the database.

- `SELECT` is usually the first word in an SQL statement. Most SQL statements are either `SELECT` or SELECT…INTO statements.

- The minimum syntax for a `SELECT` statement is:

    ```     SELECT fields FROM table```

- You can use an asterisk (*) to select all fields in a table. The following example selects all of the fields in the Employees table.

- [Microsoft Documentation](https://learn.microsoft.com/en-us/office/client-developer/access/desktop-database-reference/select-statement-microsoft-access-sql)

In [ ]:
SELECT [predicate] { * | table.* | [table.]field1 [AS alias1] [, [table.]field2 [AS alias2] [, …]]} 
FROM tableexpression [, …] [IN externaldatabase] 
[WHERE… ] 
[GROUP BY… ] 
[HAVING… ] 
[ORDER BY… ] 
[WITH OWNERACCESS OPTION]

- If a field name is included in more than one table in the `FROM` clause, precede it with the table name and the . (dot) operator.

In [ ]:
SELECT Employees.Department, Employees.SupvName 
FROM Employees 
WHERE Employees.Department = "X";

## 🔍 SELECT INTO Statement

- [Microsoft Documentaion](https://learn.microsoft.com/en-us/office/client-developer/access/desktop-database-reference/select-into-statement-microsoft-access-sql)

In [ ]:
SELECT field1[, field2[, …]] 
INTO newtable [IN externaldatabase] 
FROM source

## 🔍 Sub Queries

- You can use a subquery instead of an expression in the field list of a `SELECT `statement or in a `WHERE` or `HAVING` clause. 

- In a subquery, you use a `SELECT` statement to provide a set of one or more specific values to evaluate in the `WHERE` or `HAVING` clause expression.

- Use the `ANY` or `SOME` predicate, which are synonymous, to retrieve records in the main query that satisfy the comparison with any records retrieved in the subquery. 

- [Microsoft Documentation](https://learn.microsoft.com/en-us/office/client-developer/access/desktop-database-reference/sql-subqueries-microsoft-access-sql)

In [ ]:
SELECT * FROM Products 
WHERE UnitPrice > ANY 
(SELECT UnitPrice FROM OrderDetails 
WHERE Discount >= .25);

- Use the `ALL` predicate to retrieve only those records in the main query that satisfy the comparison with all records retrieved in the subquery. 

- Use the `IN` predicate to retrieve only those records in the main query for which some record in the subquery contains an equal value. 

In [ ]:
SELECT * FROM Products 
WHERE ProductID IN 
(SELECT ProductID FROM OrderDetails 
WHERE Discount >= .25);

- Conversely, you can use `NOT IN` to retrieve only those records in the main query for which no record in the subquery contains an equal value.

- Use the `EXISTS` predicate (with the optional `NOT` reserved word) in true/false comparisons to determine whether the subquery returns any records.

- You can also use table name aliases in a subquery to refer to tables listed in a `FROM` clause outside the subquery.

In [ ]:
SELECT LastName,
FirstName, Title, Salary 
FROM Employees AS T1 
WHERE Salary >= (SELECT Avg(Salary) 
FROM Employees 
WHERE T1.Title = Employees.Title) Order by Title;

## ⬆️ UPDATE Statement

- [Microsoft Documentation](https://learn.microsoft.com/en-us/office/client-developer/access/desktop-database-reference/update-statement-microsoft-access-sql)

In [ ]:
UPDATE Orders 
SET OrderAmount = OrderAmount * 1.1, 
Freight = Freight * 1.03 
WHERE ShipCountry = 'UK';

## 🛠️ INSERT Statement

- You can use the `INSERT INTO` statement to add a single record to a table using the single-record append query syntax as shown above. In this case, your code specifies the name and value for each field of the record. 

- You must specify each of the fields of the record that a value is to be assigned to and a value for that field. 

- When you do not specify each field, the default value or Null is inserted for missing columns. 

- Records are added to the end of the table.

- You can also use `INSERT INTO` to append a set of records from another table or query by using the `SELECT` … `FROM` clause as shown above in the multiple-record append query syntax. In this case, the `SELECT` clause specifies the fields to append to the specified target table.

- The source or target table may specify a table or a query. If a query is specified, the Microsoft Access database engine appends records to any and all tables specified by the query.

- `INSERT INTO` is optional but when included, precedes the `SELECT` statement.

- If your destination table contains a primary key, make sure you append unique, non-Null values to the primary key field or fields; if you do not, the Microsoft Access database engine will not append the records.

- If you append records to a table with an `AutoNumber` field and you want to renumber the appended records, do not include the `AutoNumber` field in your query. Do include the `AutoNumber` field in the query if you want to retain the original values from the field.

- Use the `IN` clause to append records to a table in another database.

- To create a new table, use the `SELECT...INTO` statement instead to create a make-table query.

- To find out which records will be appended before you run the append query, first execute and view the results of a select query that uses the same selection criteria.

- An append query copies records from one or more tables to another. The tables that contain the records you append are not affected by the append query.

- Instead of appending existing records from another table, you can specify the value for each field in a single new record using the `VALUES` clause. If you omit the field list, the `VALUES` clause must include a value for every field in the table; otherwise, the` INSERT` operation will fail. Use an additional `INSERT INTO` statement with a `VALUES` clause for each additional record you want to create.

- [Microsoft Documentation](https://learn.microsoft.com/en-us/office/client-developer/access/desktop-database-reference/insert-into-statement-microsoft-access-sql)

#### Single-record

In [ ]:
INSERT INTO table_name [(field1[, field2[, …]])] 
VALUES (value1[, value2[, …])

#### Multiple-record

In [ ]:
INSERT INTO table_name [(field1[, field2[, …]])] [IN externaldatabase] 
SELECT [source.]field1[, field2[, …] 
FROM tableexpression

## 🧠 Execution Order


- The **Jet/ACE engine** processes statements in a specific **logical order** that determines how results are built.

- Understanding this sequence explains many Access “mysteries,” such as why aliases aren’t recognized in the `WHERE` clause or why totals queries require the `HAVING` clause.

#### Logical Order of Execution

| Step  | Clause           | Description                                   |
| ----- | ---------------- | --------------------------------------------- |
| **1** | `FROM`           | Load tables and perform joins or subqueries.  |
| **2** | `WHERE`          | Filter individual rows (row-level filtering). |
| **3** | `GROUP BY`       | Group the remaining rows into categories.     |
| **4** | `HAVING`         | Filter groups based on aggregate results.     |
| **5** | `SELECT`         | Return specific columns or expressions.       |
| **6** | `ORDER BY`       | Sort the final result set.                    |
| **7** | `TOP / DISTINCT` | Apply record limits or remove duplicates.     |

### Example: Department Salary Analysis

In [ ]:
SELECT Department, AVG(Salary) AS AvgSalary
FROM Employees
WHERE HireDate >= #1/1/2020#
GROUP BY Department
HAVING AVG(Salary) > 85000
ORDER BY AvgSalary DESC;

**Execution flow:**

1. **`FROM`** — Access retrieves all records from `Employees`.

2. **`WHERE`** — Filters employees hired after January 1, 2020.

3. **`GROUP BY`** — Groups remaining employees by department.

4. **`HAVING`** — Keeps only groups with an average salary above $85,000.

5. **`SELECT`** — Produces two columns: `Department` and the calculated `AvgSalary`.

6. **`ORDER BY`** — Sorts results from highest to lowest average salary.

7. **`TOP`** (if present) — Would then limit the number of rows returned.


## ⚠️ Access Nuances

- **Access executes JOINs first**, even before evaluating `WHERE` filters.

- This means row combinations are formed before filtering — an important distinction when working with outer joins.

- **Aliases defined in `SELECT` cannot be used in `WHERE`** because the `WHERE` clause executes first.

-  You can use aliases in `ORDER BY` since it executes last.

- **`HAVING` is the only clause** that can reference aggregate functions such as `SUM()` or `AVG()`.

- **`DISTINCT` and `TOP`** are applied *after* ordering — which is why applying `TOP 10` to an unordered query can yield inconsistent results.

- **Totals Queries in Design View** correspond exactly to the `GROUP BY` → `HAVING` stages.

#### Why It Matters

| Common Confusion                                      | Explanation                                                             |
| ----------------------------------------------------- | ----------------------------------------------------------------------- |
| “Why does Access say my alias doesn’t exist?”         | Because the alias is created in `SELECT`, which runs after `WHERE`.     |
| “Why can’t I filter averages in `WHERE`?”               | Aggregates don’t exist yet; you must use `HAVING`.                      |
| “Why does changing `JOIN` type change my record count?” | Access executes joins before filtering, affecting which rows qualify.   |
| “Why does `TOP` 10 behave differently each run?”        | Without `ORDER BY`, Access picks arbitrary rows — add explicit sorting. |

## 🧠 Logical vs. Physical Processing

- This order represents the **logical** flow of SQL — the conceptual sequence the Jet/ACE engine uses.

- Internally, Access may reorder or optimize steps for performance (e.g., pushing filters earlier, using indexes, or caching joined tables).

- Understanding the logical sequence is crucial for writing queries that behave predictably.

### Quick Reference Diagram

```
┌────────────────────────────────────┐
│ FROM → WHERE → GROUP BY → HAVING   │
│ → SELECT → ORDER BY → TOP/DISTINCT │
└────────────────────────────────────┘
```


## 📝 SQL Clauses






| Clause     | Purpose                                                 | Notes                                        |
| ---------- | ------------------------------------------------------- | -------------------------------------------- |
| `SELECT`   | Specifies which columns (fields) to return.             | You can also include calculated expressions. |
| `FROM`     | Indicates which table(s) to read from.                  | Supports joins and subqueries.               |
| `WHERE`    | Filters rows based on a condition.                      | Optional; works before grouping.             |
| `ORDER BY` | Sorts results ascending (`ASC`) or descending (`DESC`). | Access defaults to ascending.                |

- If you omit the `WHERE` clause, Access returns all records in the table — similar to “Select All”.



## 📅 Data Types & Literals 



- Access SQL uses a simple but strict system for data representation.

| Data Type     | Example          | Notes                                       |
| ------------- | ---------------- | ------------------------------------------- |
| **Text**      | `"Smith"`        | Strings use double quotes or single quotes. |
| **Number**    | `42`, `3.14`     | No quotes needed.                           |
| **Date/Time** | `#1/1/2025#`     | Date literals **must** be enclosed in `#`.  |
| **Boolean**   | `True` / `False` | Stored internally as -1 and 0.              |



- Access always interprets dates in **U.S. format (MM/DD/YYYY)**, regardless of regional settings.

- If your system uses a different locale, still write `#12/31/2025#` (not `#31/12/2025#`).

#### Example:


In [ ]:
SELECT * FROM Orders
WHERE OrderDate >= #1/1/2025# AND Shipped = True;


## 🔍 WHERE Clause


- The `WHERE` clause refines which records appear in your results.

#### Comparison Operators

| Operator             | Description | Example                    |
| -------------------- | ----------- | -------------------------- |
| `=`                  | Equal to    | `WHERE City = "Boston"`    |
| `<>`                 | Not equal   | `WHERE Department <> "IT"` |
| `<`, `>`, `<=`, `>=` | Comparison  | `WHERE Salary >= 60000`    |

#### Combining Conditions


> Logical operators `AND`, `OR`, and `NOT` combine multiple conditions.



In [ ]:
SELECT * FROM Employees
WHERE Department = "Finance"
  AND Salary > 80000;


## ✨ LIKE Pattern Matching 


- Unlike most SQL dialects, Access uses `*` and `?` as wildcards (not `%` and `_`).

> Returns all cities beginning with “New” (e.g., *New York*, *Newark*).



- An exclamation mark `!` at the beginning of charlist means that a match is made if any character except those in charlist are found in expression. When used outside brackets, the exclamation mark matches itself.

- You can use the hyphen `-` either at the beginning (after an exclamation mark if one is used) or at the end of charlist to match itself. In any other location, the hyphen identifies a range of ANSI characters.

- When you specify a range of characters, the characters must appear in ascending sort order (A-Z or 0-100). `[A-Z]` is a valid pattern, but [Z-A] is not.

- The character sequence `[` `]` is ignored; it is considered to be a zero-length string ("").

In [ ]:
SELECT * FROM Customers
WHERE City LIKE "New*";


## ❓ Null Checks


- Because `NULL` represents “no value,” comparisons like `= NULL` will fail.

- Use `IS NULL` or `IS NOT NULL`:



In [ ]:

SELECT * FROM Orders
WHERE ShippedDate IS NULL;

## 💾 Sorting & Aliases

- Sorting results makes data easier to analyze or present in reports.

- `AS` assigns a friendly alias to a column name.

- By default, `ORDER BY` sorts ascending; append `DESC` for descending order.



In [ ]:

SELECT LastName AS EmployeeLast, FirstName AS EmployeeFirst
FROM Employees
ORDER BY EmployeeLast ASC;



## 🛠️ Table Aliases

- Table aliases shorten long table names, especially in joins:



In [ ]:


SELECT e.FirstName, e.LastName, d.DepartmentName
FROM Employees AS e
INNER JOIN Departments AS d
ON e.DepartmentID = d.DepartmentID;



## 🧮 Calculated Fields and Built-In Functions

Access lets you compute values directly in queries using expressions and built-in functions.

### Example: Calculated Field

In [ ]:
SELECT FirstName, LastName, Salary, Salary * 1.05 AS NewSalary
FROM Employees;

Creates a new calculated column named **NewSalary**.

### Common Built-In Functions

| Category        | Function                              | Example                              | Description                      |
| --------------- | ------------------------------------- | ------------------------------------ | -------------------------------- |
| **String**      | `LEFT(text, n)`                       | `LEFT(LastName, 3)`                  | Returns leftmost `n` characters. |
|                 | `LEN(text)`                           | `LEN(LastName)`                      | Counts string length.            |
| **Date/Time**   | `DateAdd(interval, n, date)`          | `DateAdd("m", 3, OrderDate)`         | Adds months, days, or years.     |
|                 | `Now()`                               | –                                    | Current date and time.           |
| **Math**        | `Round(x, n)`                         | `Round(Salary, 0)`                   | Rounds numbers.                  |
| **Conditional** | `IIf(condition, truepart, falsepart)` | `IIf(Salary>100000,"High","Normal")` | Inline conditional expression.   |

These expressions can appear in any `SELECT`, `WHERE`, or `ORDER BY` clause.





## 🔗 Joins 

- Relational databases store related data across multiple tables.
**Joins** merge those tables logically when querying.

### INNER JOIN

- Returns only matching records from both tables.

- [Microsoft Documentation](https://learn.microsoft.com/en-us/office/client-developer/access/desktop-database-reference/inner-join-operation-microsoft-access-sql)


In [ ]:
SELECT e.FirstName, e.LastName, d.DepartmentName
FROM Employees AS e
INNER JOIN Departments AS d
ON e.DepartmentID = d.DepartmentID;

### LEFT JOIN

- Includes all records from the left table, even if there’s no match in the right.

In [ ]:
SELECT c.CustomerName, o.OrderID
FROM Customers AS c
LEFT JOIN Orders AS o
ON c.CustomerID = o.CustomerID;


### RIGHT JOIN

Opposite of `LEFT JOIN` — includes all records from the right table.



### Notes on Access Join Syntax

* The Query Designer uses **visual join lines**; switching to SQL View shows equivalent `JOIN` statements.

* Access supports nested joins but may reformat them automatically.

* Unlike SQL Server, Access does **not** support `FULL OUTER JOIN` directly — use a `UNION` of `LEFT` and `RIGHT` joins.

* [Microsoft Documentation](https://learn.microsoft.com/en-us/office/client-developer/access/desktop-database-reference/left-join-right-join-operations-microsoft-access-sql)


## 📊 Grouping and Aggregation

- Grouping lets you compute totals, averages, or counts across categories.

- Using the SQL aggregate functions, you can determine various statistics on sets of values. 

- You can use these functions in a query and aggregate expressions in the SQL property of a `QueryDef` object or when creating a Recordset object based on an SQL query.


In [ ]:
SELECT Department, AVG(Salary) AS AvgSalary
FROM Employees
GROUP BY Department
HAVING AVG(Salary) > 80000;

* `GROUP BY` defines how rows are grouped.

* Aggregate functions ( `SUM`, `AVG`, `COUNT`, `MIN`, `MAX` ) summarize data.

* `HAVING` filters grouped results (while `WHERE` filters individual rows).

- [Microsoft Documentation](https://learn.microsoft.com/en-us/office/client-developer/access/desktop-database-reference/sql-aggregate-functions-sql)


## 🧩 Subqueries

Subqueries allow one query to feed another — useful for filters, comparisons, or calculations.

### Using IN


Example explanation:

```
        “Show employess in departments located in DC"
```


In [ ]:
SELECT FirstName, LastName
FROM Employees
WHERE DepartmentID IN
    (SELECT DepartmentID FROM Departments WHERE Location = "HQ");

### Using EXISTS

- Access supports nested subqueries up to several levels deep, but they can become slow on large datasets — use joins where possible.


In [ ]:
SELECT CustomerName
FROM Customers AS c
WHERE EXISTS
    (SELECT * FROM Orders AS o WHERE o.CustomerID = c.CustomerID);

## ⚡ Action Queries (Data Modification)

- Action queries change data or create new tables.

### INSERT INTO

In [ ]:
INSERT INTO Employees (FirstName, LastName, Department)
VALUES ("Jane", "Doe", "Finance");

### UPDATE

In [ ]:
UPDATE Employees
SET Salary = Salary * 1.1
WHERE Department = "Sales";

### DELETE

In [ ]:
DELETE FROM Orders
WHERE OrderDate < #1/1/2020#;


### MAKE-TABLE

- Creates a new table with results of a query.

In [ ]:
SELECT * INTO HighEarners
FROM Employees
WHERE Salary > 100000;

- Action queries are powerful — always back up before running them.


## 🧭 Parameter Queries

- Parameter queries prompt users for input dynamically.

- Access will display input boxes for `[Enter Start Date:]` and `[Enter End Date:]`.

- For queries that you run regularly, you can use a `PARAMETERS` declaration to create a parameter query. A parameter query can help automate the process of changing query criteria. 

- With a parameter query, your code will need to provide the parameters each time the query is run.

- The `PARAMETERS` declaration is optional but when included precedes any other statement, including `SELECT`.

- If the declaration includes more than one parameter, separate them with commas

- [Microsoft Documentation](https://learn.microsoft.com/en-us/office/client-developer/access/desktop-database-reference/parameters-declaration-microsoft-access-sql)


In [ ]:
SELECT * FROM Orders
WHERE OrderDate BETWEEN [Enter Start Date:] AND [Enter End Date:];

- `PARAMETERS` declaration and use

In [ ]:
PARAMETERS [Low price] Currency, 
[Beginning date] DateTime; 
SELECT OrderID, OrderAmount
FROM Orders 
WHERE OrderAmount > [Low price] 
AND OrderDate >= [Beginning date];

- [Microsoft Documentation](https://learn.microsoft.com/en-us/office/client-developer/access/desktop-database-reference/parameters-declaration-microsoft-access-sql)

### Executing Parameters via VBA


In [ ]:
Dim qd As DAO.QueryDef, rs As DAO.Recordset
Set qd = CurrentDb.QueryDefs("qrySalesByDate")
qd.Parameters("[Enter Start Date:]") = #1/1/2025#
qd.Parameters("[Enter End Date:]") = #1/31/2025#
Set rs = qd.OpenRecordset()

## 🧮 Domain Aggregate Functions

These functions retrieve calculated values directly from tables or queries — often used in VBA or form controls.

| Function  | Description            | Example                                   |
| --------- | ---------------------- | ----------------------------------------- |
| `DLookup` | Returns a single value | `DLookup("Salary","Employees","ID=5")`    |
| `DSum`    | Sums field values      | `DSum("Amount","Orders","CustomerID=7")`  |
| `DCount`  | Counts records         | `DCount("*","Customers","City='Boston'")` |




## 📊 Crosstab Queries (TRANSFORM)

- Crosstab queries summarize data across two dimensions, similar to Excel pivot tables.

In [ ]:
TRANSFORM Sum(Amount) AS TotalSales
SELECT Region
FROM Sales
GROUP BY Region
PIVOT Year;

- This produces a table with **Region** as rows, **Year** as columns, and total sales in the cells.

In [ ]:
SELECT Name, City FROM Customers_US
UNION ALL
SELECT Name, City FROM Customers_Canada;


## 🧱 UNION Queries

- Combine results from multiple queries with identical structures.

- Use `UNION` to remove duplicates or `UNION ALL` to include them.

In [ ]:
TABLE [New Accounts] UNION ALL 
SELECT * 
FROM Customers 
WHERE OrderAmount > 1000;


## 💻 Integrating SQL with VBA

- VBA turns Access into a programmable database system.

### Executing Action Queries


In [ ]:
DoCmd.RunSQL "DELETE FROM TempData WHERE EntryDate < Date();"

### Working with Recordsets

In [ ]:
Dim rs As DAO.Recordset
Dim sql As String
sql = "SELECT * FROM Employees WHERE Department='Finance';"
Set rs = CurrentDb.OpenRecordset(sql)
Do While Not rs.EOF
    Debug.Print rs!FirstName & " " & rs!LastName
    rs.MoveNext
Loop
rs.Close

### Dynamic SQL Assembly

In [ ]:
Dim startDate As Date, endDate As Date
startDate = #1/1/2025#: endDate = #1/31/2025#
sql = "SELECT * FROM Orders WHERE OrderDate BETWEEN #" & _
       Format(startDate, "mm/dd/yyyy") & "# AND #" & Format(endDate, "mm/dd/yyyy") & "#;"
Set rs = CurrentDb.OpenRecordset(sql)

## ⚠️ Common Pitfalls and Best Practices

| Issue                     | Recommendation                                              |
| ------------------------- | ----------------------------------------------------------- |
| **Reserved Words**        | Use square brackets around names like `[Date]` or `[Name]`. |
| **Spaces in Field Names** | Always use `[Field Name]` notation.                         |
| **Wildcard Confusion**    | Use `*` and `?` — not `%` and `_`.                          |
| **Date Literals**         | Always use `#MM/DD/YYYY#`.                                  |
| **Query Performance**     | Avoid `SELECT *`; specify columns explicitly.               |
| **Data Validation**       | Use `WHERE` and parameter checks to prevent errors.         |




# 🧾 Quick Reference Tables



## Data Type Mapping

| Access Type      | SQL Equivalent | Example        |
| ---------------- | -------------- | -------------- |
| Short Text       | VARCHAR        | `"Hello"`      |
| Long Text        | MEMO           | long notes     |
| Number (Integer) | INT            | `42`           |
| Currency         | MONEY          | `12.99`        |
| Date/Time        | DATETIME       | `#2025-11-06#` |
| Yes/No           | BOOLEAN        | `True`         |


## ✨ Common Function Summary

| Category       | Function                     | Description                     |
| -------------- | ---------------------------- | ------------------------------- |
| **Text**       | `UCase(text)`                | Converts to uppercase           |
|                | `Trim(text)`                 | Removes leading/trailing spaces |
| **Date**       | `Date()`, `Now()`            | Current date/time               |
|                | `DateDiff(interval, d1, d2)` | Difference between two dates    |
| **Math**       | `Abs(x)`                     | Absolute value                  |
|                | `Rnd()`                      | Random number                   |
| **Logic**      | `IIf(cond, t, f)`            | Inline conditional              |
| **Conversion** | `CInt()`, `CDate()`          | Type conversion                 |

## 🧩 Summary

Access SQL is a **relational query language** that brings professional-grade data querying to the desktop.
When paired with VBA, it allows you to:

* Automate reports and data entry

* Build parameterized dashboards

* Perform analysis comparable to SQL Server or MySQL — on a smaller scale

- Understanding Access SQL not only deepens your Access skills but also prepares you for transitioning to enterprise databases like SQL Server.



### 📚 Further Reading

* **Microsoft Docs:** [ACE SQL Reference](https://learn.microsoft.com/en-us/previous-versions/office/developer/office-2010/ff965871(v=office.14)#architecture-of-the-microsoft-access-engine)

* **DAO Language Reference**

* **Allen Browne’s Access Tips** (excellent real-world examples)


# 🗂️ Keyword Reference

## 📌 Data Query 


| Keyword       | Description                                     | Example Usage                                                              |
| ------------- | ----------------------------------------------- | -------------------------------------------------------------------------- |
| `SELECT`      | Retrieves fields from tables or queries         | `SELECT FirstName FROM Employees;`                                         |
| `FROM`        | Specifies the table or query source             | `SELECT * FROM Orders FROM Orders;`                                        |
| `WHERE`       | Filters rows                                    | `SELECT * FROM Orders WHERE Amount > 1000;`                                |
| `ORDER BY`    | Sorts result set                                | `SELECT * FROM Clients ORDER BY LastName;`                                 |
| `GROUP BY`    | Groups rows for aggregate functions             | `SELECT Dept, COUNT(*) FROM Employees GROUP BY Dept;`                      |
| `HAVING`      | Post-aggregation filter                         | `SELECT Dept, COUNT(*) FROM Employees GROUP BY Dept HAVING COUNT(*) > 10;` |
| `DISTINCT`    | Removes duplicates                              | `SELECT DISTINCT Country FROM Suppliers;`                                  |
| `DISTINCTROW` | Removes duplicate rows based on *entire record* | `SELECT DISTINCTROW LastName FROM Employees;`                              |
| `AS`          | Assigns alias to fields or tables               | `SELECT Salary AS AnnualPay FROM Employees;`                               |
| `TOP`         | Returns first N rows                            | `SELECT TOP 10 * FROM Sales;`                                              |
| `PERCENT`     | Used with TOP for percentage                    | `SELECT TOP 10 PERCENT * FROM Sales;`                                      |
| `IN`          | Filters list membership                         | `SELECT * FROM Orders WHERE State IN ('CA','TX');`                         |
| `BETWEEN`     | Range comparison                                | `SELECT * FROM Orders WHERE Amount BETWEEN 100 AND 500;`                   |
| `LIKE`        | Pattern matching                                | `SELECT * FROM Employees WHERE LastName LIKE 'S*';`                        |
| `IS NULL`     | Tests for null                                  | `SELECT * FROM Tasks WHERE DueDate IS NULL;`                               |
| `IS NOT NULL` | Tests for non-null                              | `SELECT * FROM Tasks WHERE DueDate IS NOT NULL;`                           |
| `ALL`         | Compares with all values                        | `Amount > ALL (SELECT Amount FROM Orders);`                                |
| `ANY`         | Compares with any value                         | `Amount > ANY (SELECT Amount FROM Orders);`                                |
| `SOME`        | Same as ANY                                     |                                                                            |


## 🔗 Join & Relationship

| Keyword                           | Description                   | Example                                        |
| --------------------------------- | ----------------------------- | ---------------------------------------------- |
| `JOIN`                            | General join keyword          |                                                |
| `INNER JOIN`                      | Rows that match both sides    | `SELECT * FROM A INNER JOIN B ON A.ID = B.ID;` |
| `LEFT JOIN` / `LEFT OUTER JOIN`   | All left rows + matches       | `SELECT * FROM A LEFT JOIN B ON ...`           |
| `RIGHT JOIN` / `RIGHT OUTER JOIN` | All right rows + matches      | *(Supported by ACE SQL)*                       |
| `FULL JOIN`                       | ❌ **Not supported** in Access | Use `UNION` of left + right joins              |
| `ON`                              | Join condition                | `...ON Orders.CustID = Customers.CustID`       |


## 🧮  Aggregation & Statistical

| Keyword  | Description          | Example                            |
| -------- | -------------------- | ---------------------------------- |
| `COUNT`  | Count rows           | `COUNT(*)`                         |
| `SUM`    | Sum numeric values   | `SUM(Amount)`                      |
| `AVG`    | Average              | `AVG(Score)`                       |
| `MIN`    | Minimum value        | `MIN(Price)`                       |
| `MAX`    | Maximum value        | `MAX(Price)`                       |
| `STDEV`  | Standard deviation   | `SELECT STDEV(Amount) FROM Sales;` |
| `STDEVP` | Population std. dev. |                                    |
| `VAR`    | Statistical variance |                                    |
| `VARP`   | Population variance  |                                    |


## 🧱 Data Modification

| Keyword       | Description            | Example                                        |
| ------------- | ---------------------- | ---------------------------------------------- |
| `INSERT INTO` | Inserts rows           | `INSERT INTO Clients (Name) VALUES ('Bob');`   |
| `VALUES`      | Value list with INSERT |                                                |
| `UPDATE`      | Modifies rows          | `UPDATE Employees SET Salary = 60000 WHERE...` |
| `SET`         | Assigns values         |                                                |
| `DELETE`      | Deletes rows           | `DELETE FROM Orders WHERE ID = 10;`            |


## 🏗️  Table Creation & DDL

| Keyword                   | Description               | Example                                          |
| ------------------------- | ------------------------- | ------------------------------------------------ |
| `CREATE TABLE`            | Creates new table         | `CREATE TABLE Test (ID COUNTER, Name TEXT(50));` |
| `DROP TABLE`              | Deletes table             | `DROP TABLE TempData;`                           |
| `ALTER TABLE`             | Modifies structure        | `ALTER TABLE A ADD COLUMN Age BYTE;`             |
| `ADD COLUMN`              | Adds a field              |                                                  |
| `DROP COLUMN`             | Removes a field           |                                                  |
| `CONSTRAINT`              | Creates constraint        |                                                  |
| `PRIMARY KEY`             | Table primary key         | `CONSTRAINT PK_Order PRIMARY KEY (OrderID)`      |
| `FOREIGN KEY`             | Link to another table     |                                                  |
| `REFERENCES`              | Defines FK target         |                                                  |
| `NOT NULL`                | Disallow nulls            |                                                  |
| `WITH OWNERACCESS OPTION` | Elevated permissions flag | *(Jet-specific)*                                 |


## 🧰 Query & Procedure

| Keyword      | Description                 | Example                                |
| ------------ | --------------------------- | -------------------------------------- |
| `PARAMETERS` | Defines query parameters    | `PARAMETERS [pID] LONG; SELECT ...`    |
| `TRANSFORM`  | Starts a crosstab query     | `TRANSFORM SUM(Amount) SELECT ...`     |
| `PIVOT`      | Pivot column                | `PIVOT MonthName;`                     |
| `INTO`       | Output into new table       | `SELECT * INTO BackupTable FROM Data;` |
| `UNION`      | Combines results (dedup)    | `SELECT ... UNION SELECT ...`          |
| `UNION ALL`  | Combines results (no dedup) |                                        |


## 🔤 Data Type

| Keyword              | Description            |
| -------------------- | ---------------------- |
| `SHORT` / `BYTE`     | 0–255                  |
| `INTEGER` / `LONG`   | Standard numeric types |
| `SINGLE`             | Floating point         |
| `DOUBLE`             | Floating point         |
| `MONEY` / `CURRENCY` | Currency               |
| `TEXT(n)`            | Up to 255 chars        |
| `CHAR(n)`            | ANSI char              |
| `VARCHAR(n)`         | Alias for TEXT         |
| `MEMO` / `LONGTEXT`  | Long text              |
| `DATETIME`           | Date/time field        |
| `YESNO`              | Boolean                |
| `COUNTER`            | AutoNumber             |
| `DECIMAL`            | Decimal type           |
| `NUMERIC`            | Alias                  |
| `GUID`               | Unique identifier      |


## ⚙️ Logical & Comparison Operators

| Keyword / Operator      | Description    | Example            |
| ----------------------- | -------------- | ------------------ |
| `AND`                   | Logical AND    | `WHERE A AND B`    |
| `OR`                    | Logical OR     | `WHERE A OR B`     |
| `NOT`                   | Negation       | `WHERE NOT Active` |
| `=`                     | Equals         |                    |
| `<>`                    | Not equal      |                    |
| `<` / `>` / `<=` / `>=` | Comparison     |                    |
| `IN`                    | Set membership |                    |
| `BETWEEN`               | Range          |                    |
| `LIKE`                  | Pattern        |                    |


## 📤  Miscellaneous Access-Specific

| Keyword                   | Description                        |
| ------------------------- | ---------------------------------- |
| `WITH OWNERACCESS OPTION` | Execute with elevated privileges   |
| `DISTINCTROW`             | Record-level duplicate suppression |
| `BINARY`                  | Binary comparisons in LIKE         |
| `ALL` / `ANY` / `SOME`    | Quantifier operators               |
| `EXISTS`                  | Subquery existence test            |


# 🗂️ Function Reference

## 📌 Text & String  

| Function                         | Purpose                      | Example                       |
| -------------------------------- | ---------------------------- | ----------------------------- |
| `LEFT(string, length)`           | Returns leftmost characters  | `LEFT([LastName], 1)`         |
| `RIGHT(string, length)`          | Returns rightmost characters | `RIGHT([Phone], 4)`           |
| `MID(string, start, length)`     | Returns substring            | `MID([Code], 2, 3)`           |
| `LEN(string)`                    | Returns string length        | `LEN([Address])`              |
| `INSTR(start, string1, string2)` | Position of substring        | `INSTR(1, [Email], '@')`      |
| `INSTRREV(string1, string2)`     | Reverse search for substring | `INSTRREV([Path], '\')`       |
| `LCASE(string)`                  | Converts to lowercase        | `LCASE([City])`               |
| `UCASE(string)`                  | Converts to uppercase        | `UCASE([City])`               |
| `CHR(number)`                    | Character from ASCII         | `CHR(10)`                     |
| `ASC(char)`                      | ASCII code of char           | `ASC("A")`                    |
| `STRCOMP(a, b, compare)`         | Compares two strings         | `STRCOMP([A],[B],0)`          |
| `REPLACE(expr, find, replace)`   | Replace substring            | `REPLACE([Text],'old','new')` |
| `TRIM(string)`                   | Trim leading/trailing spaces | `TRIM([Notes])`               |
| `LTRIM(string)`                  | Trim left side               | `LTRIM([Notes])`              |
| `RTRIM(string)`                  | Trim right side              | `RTRIM([Notes])`              |
| `SPACE(n)`                       | Returns n spaces             | `SPACE(5)`                    |
| `STRING(n, char)`                | Repeat a character           | `STRING(10,'*')`              |
| `FORMAT(expr, formatString)`     | Format output                | `FORMAT([Price], "Currency")` |


## 🧮 Numeric & Mathematical

| Function                | Purpose                    | Example              |
| ----------------------- | -------------------------- | -------------------- |
| `ABS(number)`           | Absolute value             | `ABS([Balance])`     |
| `FIX(number)`           | Integer part (toward zero) | `FIX(-5.9)`          |
| `INT(number)`           | Integer part (floor)       | `INT(5.9)`           |
| `SGN(number)`           | Sign (–1, 0, 1)            | `SGN([Delta])`       |
| `ROUND(number, digits)` | Round                      | `ROUND([Amount], 2)` |
| `SQR(number)`           | Square root                | `SQR(81)`            |
| `EXP(number)`           | e^x                        | `EXP([x])`           |
| `LOG(number)`           | Natural log                | `LOG([Value])`       |
| `RND()`                 | Random number              | `RND()`              |
| `RND(-n)`               | Repeatable random seed     | `RND(-1)`            |
| `SIN(x)`                | Sine                       | `SIN([Angle])`       |
| `COS(x)`                | Cosine                     | `COS([Angle])`       |
| `TAN(x)`                | Tangent                    | `TAN([Angle])`       |
| `ATN(x)`                | Arctan                     | `ATN([Value])`       |


## 🕒 Date & Time

| Function                          | Purpose                     | Example                        |
| --------------------------------- | --------------------------- | ------------------------------ |
| `DATE()`                          | Returns current date        | `DATE()`                       |
| `TIME()`                          | Returns current time        | `TIME()`                       |
| `NOW()`                           | Current date/time           | `NOW()`                        |
| `YEAR(date)`                      | Returns year                | `YEAR([HireDate])`             |
| `MONTH(date)`                     | Returns month (1–12)        | `MONTH([OrderDate])`           |
| `DAY(date)`                       | Returns day (1–31)          | `DAY([Birthdate])`             |
| `HOUR(time)`                      | Returns hour                | `HOUR([Timestamp])`            |
| `MINUTE(time)`                    | Returns minutes             | `MINUTE([Timestamp])`          |
| `SECOND(time)`                    | Returns seconds             | `SECOND([Timestamp])`          |
| `WEEKDAY(date, firstDay)`         | Day of week                 | `WEEKDAY([Date],1)`            |
| `WEEKDAYNAME(index)`              | Day name                    | `WEEKDAYNAME(WEEKDAY(Date()))` |
| `MONTHNAME(month)`                | Month name                  | `MONTHNAME(MONTH(Date()))`     |
| `DATEADD(interval, number, date)` | Add/subtract date component | `DATEADD("d",7,[Start])`       |
| `DATEDIFF(interval, d1, d2)`      | Difference between dates    | `DATEDIFF("m",[Start],[End])`  |
| `DATEDIFF("ww",...)`              | Week difference             |                                |
| `DATESERIAL(y, m, d)`             | Build date                  | `DATESERIAL(2025, 1, 1)`       |
| `TIMESERIAL(h, m, s)`             | Build time                  | `TIMESERIAL(12, 30, 0)`        |
| `DATEPART(interval, date)`        | Extracts part of date       | `DATEPART("q",[Date])`         |
| `DATEVALUE(string)`               | Convert to date             | `DATEVALUE("1/1/2025")`        |
| `TIMEVALUE(string)`               | Convert to time             | `TIMEVALUE("08:30")`           |
| `CVDATE(string)`                  | Convert to date             | `CVDATE("2025-01-01")`         |


## 📊  SQL Aggregation

| Function        | Purpose              | Example            |
| --------------- | -------------------- | ------------------ |
| `COUNT(field)`  | Count non-null rows  | `COUNT([OrderID])` |
| `COUNT(*)`      | Count all records    | `COUNT(*)`         |
| `SUM(field)`    | Sum values           | `SUM([Amount])`    |
| `AVG(field)`    | Average              | `AVG([Score])`     |
| `MIN(field)`    | Minimum              | `MIN([HireDate])`  |
| `MAX(field)`    | Maximum              | `MAX([HireDate])`  |
| `STDEV(field)`  | Standard deviation   | `STDEV([Amount])`  |
| `STDEVP(field)` | Population std. dev. |                    |
| `VAR(field)`    | Variance             | `VAR([Amount])`    |
| `VARP(field)`   | Population variance  |                    |


## 🏛️  Domain Aggregate

| Function                          | Purpose              | Example                                |
| --------------------------------- | -------------------- | -------------------------------------- |
| `DLOOKUP(expr, domain, criteria)` | Look up a value      | `DLOOKUP("Salary","Employees","ID=5")` |
| `DCOUNT(expr, domain, criteria)`  | Count rows           | `DCOUNT("*","Orders","State='CA'")`    |
| `DSUM(expr, domain, criteria)`    | Sum values           | `DSUM("Amount","Orders","CustID=10")`  |
| `DAVG(expr, domain, criteria)`    | Average              | `DAVG("Score","Tests","Class='A'")`    |
| `DMIN(expr, domain, criteria)`    | Minimum              |                                        |
| `DMAX(expr, domain, criteria)`    | Maximum              |                                        |
| `DVAR(expr, domain, criteria)`    | Variance             |                                        |
| `DVARP(expr, domain, criteria)`   | Population variance  |                                        |
| `DSTDEV(expr, domain, criteria)`  | Standard deviation   |                                        |
| `DSTDEVP(expr, domain, criteria)` | Population std. dev. |                                        |
| `DFIRST(expr, domain)`            | First record         | `DFIRST("Date","Orders")`              |
| `DLAST(expr, domain)`             | Last record          | `DLAST("Date","Orders")`               |


## 🔄 Conversion

| Function               | Purpose                      | Example                       |
| ---------------------- | ---------------------------- | ----------------------------- |
| `CINT(expr)`           | Convert to Integer           | `CINT([Value])`               |
| `CLNG(expr)`           | Convert to Long              | `CLNG([Value])`               |
| `CSNG(expr)`           | Single precision             | `CSNG([Value])`               |
| `CDBL(expr)`           | Double precision             | `CDBL([Value])`               |
| `CDEC(expr)`           | Decimal                      | `CDEC("12.5")`                |
| `CCUR(expr)`           | Currency                     | `CCUR("99.50")`               |
| `CSTR(expr)`           | String                       | `CSTR([Amount])`              |
| `CBOOL(expr)`          | Boolean                      | `CBOOL([Flag])`               |
| `CVAR(expr)`           | Variant                      |                               |
| `CVAR(expr)`           | Variant                      |                               |
| `CVDATE(expr)`         | Convert to date              | `CVDATE("1/1/2025")`          |
| `VAL(string)`          | Convert text to number       | `VAL("123")`                  |
| `FORMAT(expr, format)` | Converts to formatted string | `FORMAT(Date(),"yyyy-mm-dd")` |


## 🧪 Type Checking

| Function            | Purpose                                 |
| ------------------- | --------------------------------------- |
| `ISNULL(expr)`      | True if null                            |
| `ISDATE(expr)`      | True if expression is date-like         |
| `ISNUMERIC(expr)`   | True if numeric                         |
| `TYPEOF(expr)`      | *(Not supported in pure SQL; VBA only)* |
| `VARIANTTYPE(expr)` | *(VBA-only; excluded here)*             |


## 🧠 Logical

| Function                              | Purpose            | Example                                 |
| ------------------------------------- | ------------------ | --------------------------------------- |
| `IIF(condition, trueExpr, falseExpr)` | If-then-else logic | `IIF([Score]>=70,"Pass","Fail")`        |
| `SWITCH(expr1,val1,expr2,val2,...)`   | Multi-branch logic | `SWITCH([x]=1,'One',[x]=2,'Two')`       |
| `CHOOSE(index, value1, value2,...)`   | Choose by index    | `CHOOSE([Quarter],"Q1","Q2","Q3","Q4")` |
| `NZ(expr, valueIfNull)`               | Replace null       | `NZ([Discount],0)`                      |


## 🧮  Financial

| Function                                   | Purpose                        | Example |
| ------------------------------------------ | ------------------------------ | ------- |
| `FV(rate, periods, payment, presentValue)` | Future value                   |         |
| `PV(rate, periods, payment)`               | Present value                  |         |
| `NPV(rate, valueArray)`                    | Net present value              |         |
| `IRR(valueArray)`                          | Internal rate of return        |         |
| `DDB(cost, salvage, life, period)`         | Declining-balance depreciation |         |


## 💻 Miscellaneous / Utility

| Function                                 | Purpose                       | Example                       |
| ---------------------------------------- | ----------------------------- | ----------------------------- |
| `CURRENTUSER()`                          | Current Access user           | `SELECT CURRENTUSER();`       |
| `ENVELOPE()`                             | *(Legacy; not normally used)* |                               |
| `PARTITION(expr, start, stop, interval)` | Range partitioning            | `PARTITION(55,0,100,10)`      |
| `FORMAT(expr, fmt)`                      | Format values                 | `FORMAT(Date(),"Short Date")` |
